In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 5-7. その他の実用例

### 5-7-1. `sort()` のアルゴリズム指定

#### コード5-49.	ソートアルゴリズムの実装例（バイトニックソート）

In [2]:
module BitonicSorterST

# ref: https://en.wikipedia.org/wiki/Bitonic_sorter

export BitonicSort

using Base.Order: Ordering, lt

struct BitonicSortAlg <: Base.Sort.Algorithm end
const BitonicSort = BitonicSortAlg()

function Base.sort!(x::AbstractVector, lo::Integer, hi::Integer, ::BitonicSortAlg, o::Ordering)
    lo ≥ hi && return x

    fullsize::Int = hi - lo
    d = sizeof(Int) * 8 - leading_zeros(fullsize - 1)  # == ceil(Int, log(2, fullsize))

    for p = 1:d, q = 1:p
        _sort_kernel!(x, lo, hi, p, q, o)
    end
    return x
end

function _sort_kernel!(x::AbstractVector, lo, hi, p, q, o)
    # @assert p ≥ q
    halfsize_1 = Int(hi - lo) >> 0x01
    d = 1 << UInt(p - q)
    for s = 0:halfsize_1
        ioff = s & (d - 1)
        seg = lo + (s ⊻ ioff) << 0x01
        joff = q == 1 ? (2d - ioff - 1) : ioff + d
        i = seg + ioff
        j = seg + joff
        if !lt(o, x[i], x[j])
            x[i], x[j] = x[j], x[i]
        end
    end
end

end # module

Main.BitonicSorterST

#### コード5-50.	`BitonicSort` の動作例

In [3]:
using .BitonicSorterST

In [4]:
x_org = [10, 30, 11, 20, 4, 330, 21, 110];

In [5]:
sort(x_org)  # デフォルトのソートアルゴリズムでソート

8-element Vector{Int64}:
   4
  10
  11
  20
  21
  30
 110
 330

In [6]:
sort(x_org; alg=BitonicSort)  # BitonicSortでソート

8-element Vector{Int64}:
   4
  10
  11
  20
  21
  30
 110
 330

In [7]:
sort(x_org; alg=BitonicSort, lt=(>))  # BitonicSortで逆順ソート

8-element Vector{Int64}:
 330
 110
  30
  21
  20
  11
  10
   4

In [8]:
strs_org = ["Julia", "is", "fast", "dynamiccaly", "typed", "with", "multiple", "dispatch"];

In [9]:
sort(strs_org; alg=BitonicSort)

8-element Vector{String}:
 "Julia"
 "dispatch"
 "dynamiccaly"
 "fast"
 "is"
 "multiple"
 "typed"
 "with"

In [10]:
check_sorted = sort!(rand(Float64, 2^16); alg=BitonicSort);

In [11]:
issorted(check_sorted)

true

### 5-7-2. `Val{N}` によるディスパッチ

#### 仮想コード5-4. `ntuple(f, Val{N})` の実装例

```julia
# ntuple の実装例（抜粋、一部改変）
ntuple(f, ::Val{0}) = ()
ntuple(f, ::Val{1}) = (f(1),)
ntuple(f, ::Val{2}) = (f(1), f(2))
ntuple(f, ::Val{3}) = (f(1), f(2), f(3))
function ntuple(f::F, ::Val{N}) where {F,N}
    N::Int
    (N >= 0) || throw(ArgumentError(string("tuple length should be ≥ 0, got ", N)))
    ((f(i) for i = 1:N)...,)
end
```

#### コード5-51. 色々な国の挨拶を返す関数 `hello()` の例

In [12]:
hello(lang::Symbol = :en) = hello(Val(lang))

hello (generic function with 2 methods)

In [13]:
hello(::Val{:en}) = println("Hello!")

hello (generic function with 3 methods)

In [14]:
hello(::Val{:ja}) = println("こんにちは！")

hello (generic function with 4 methods)

In [15]:
hello(::Val{:zh}) = println("你好!")

hello (generic function with 5 methods)

In [16]:
hello(::Val{:ko}) = println("안녕하세요!")

hello (generic function with 6 methods)

In [17]:
hello(::Val{:fr}) = println("Bonjour!")

hello (generic function with 7 methods)

In [18]:
hello(::Val{:it}) = println("Ciao!")

hello (generic function with 8 methods)

In [19]:
hello(::Val{:po}) = println("Olá!")

hello (generic function with 9 methods)

In [20]:
function hello(::Val{lang}) where {lang}
    println("Sorry but I'm not familiar with your language ($lang).")
    println("Tell me the greeting of your native.")
end

hello (generic function with 10 methods)

In [21]:
hello()

Hello!


In [22]:
hello(:ja)

こんにちは！


In [23]:
hello(:it)

Ciao!


In [24]:
hello(:th)

Sorry but I'm not familiar with your language (th).
Tell me the greeting of your native.


In [25]:
hello(::Val{:th}) = println("สวัสดี!")

hello (generic function with 11 methods)

In [26]:
hello(:th)

สวัสดี!
